# Fichier visant à récupérer les data et à les mettre dans une table unique

In [2]:
import datetime
import mysql.connector
import numpy as np
import os
import pandas as pd 

## Connection à la base de donnée et récupération des tables sous forme de df

In [ ]:
# Connexion a la database

def connexion_db():
    mydb = mysql.connector.connect(
      host="hostname",
      user="username",
      password='password',
      database="databasename"
    )

    # Récupération des data concernant les compteurs de Rivoli et de Sebastopol (heure par heure)
    mycursor = mydb.cursor()
    mycursor.execute('SELECT * FROM bike')

    columns = [i[0] for i in mycursor.description]
    table_rows = mycursor.fetchall()

    df_borne = pd.DataFrame(table_rows,columns=columns)

    # Récupération des data concernant les jours des vacances et le jour de la semaine
    mycursor = mydb.cursor()
    mycursor.execute('SELECT * FROM calendar')

    columns = [i[0] for i in mycursor.description]
    table_rows = mycursor.fetchall()

    df_calendrier = pd.DataFrame(table_rows,columns=columns)

    # Récupération des data concernant la météo (température, humidité, nébulosité, précipitation) (Toutes les 3h)
    mycursor = mydb.cursor()
    mycursor.execute('SELECT * FROM weather')

    columns = [i[0] for i in mycursor.description]
    table_rows = mycursor.fetchall()

    df_meteo = pd.DataFrame(table_rows,columns=columns)

    # Fermeture de la connexion à la db
    mydb.close() 
    
    return(df_borne,df_meteo,df_calendrier)

## Mise en forme dans un dataframe unique

In [3]:
def get_dataset():
    
    df_borne,df_meteo,df_calendrier = connexion_db()

    # Convertion des dates dans le même format
    df_calendrier['date'] = pd.to_datetime(df_calendrier['date'],utc=True).dt.tz_localize(None)
    df_borne['date'] = pd.to_datetime(df_borne['date'],utc=True).dt.tz_localize(None)
    df_meteo.date = pd.to_datetime(df_meteo.date,format='%Y%m%d%H%M%S',utc=True).dt.tz_localize(None)

    # Transformation des données méteo en float
    df_meteo['rr1'] = df_meteo['rr1'].replace("mq",np.nan)
    df_meteo['rr3'] = df_meteo['rr3'].replace("mq",np.nan)
    df_meteo['n'] = df_meteo['n'].replace("mq",np.nan)
    df_meteo[['ff','t','u','n','rr1','rr3']] = df_meteo[['ff','t','u','n','rr1','rr3']].astype(float)

    # Selection des deux plus gros compteurs
    df_sebastopol = df_borne[df_borne['id_compteur']=='100057445-104057445'].sort_values(by='date')[['date','sum_counts']]
    df_rivoli = df_borne[df_borne['id_compteur']=='100050876-104050876'].sort_values(by='date')[['date','sum_counts']]

    # Jointure des données meteo pour le compteur Rivoli
    df_meteo = df_meteo.set_index(df_meteo.date).drop(columns='date')
    df_rivoli = df_rivoli.set_index(df_rivoli['date'])
    df_rivoli = df_rivoli.join(df_meteo).fillna(method='bfill').fillna(method='ffill').reset_index(drop = True)

    # Jointure des données meteo pour le compteur Sebastopol
    df_sebastopol = df_sebastopol.set_index(df_sebastopol['date'])
    df_sebastopol = df_sebastopol.join(df_meteo).fillna(method='bfill').fillna(method='ffill').reset_index(drop = True)

    # Jointure des données calendaires 
    df_calendrier = df_calendrier.set_index(df_calendrier.date.dt.date).drop(columns='date')
    df_rivoli = df_rivoli.set_index(df_rivoli.date.dt.date)
    df_rivoli = df_rivoli.join(df_calendrier)

    df_sebastopol = df_sebastopol.set_index(df_sebastopol.date.dt.date)
    df_sebastopol = df_sebastopol.join(df_calendrier)
    
    return(df_sebastopol,df_rivoli)